# Actinia FUTURES Demo

In [1]:
# Import Python standard library and IPython packages we need.
%load_ext dotenv
%dotenv


import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'
print(gisbase)
ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

/home/coreywhite/.local/grass83


In [6]:
!g.extension extension=ace url=https://github.com/mundialis/ace

/home/coreywhite/.local/grass83/scripts/g.extension:167: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree
Fetching <ace> from <https://github.com/mundialis/ace/archive/master.zip>
(be patient)...
Compiling...
Installing...
Updating extensions metadata file...
Updating extension modules metadata file...
Installation of <ace> successfully finished


In [2]:
def print_as_json(data):
    print(json.dumps(data, indent=2))

## Actinia Extension

In [18]:
!ace location="futures_triangle_nc" create_mapset="futures_test"

Trying to create mapset futures_test
{'accept_datetime': '2022-08-12 18:56:16.633790',
 'accept_timestamp': 1660330576.6337895,
 'api_info': {'endpoint': 'mapsetmanagementresourceadmin',
              'method': 'POST',
              'path': '/api/v3/locations/futures_triangle_nc/mapsets/futures_test',
              'request_url': 'http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test'},
 'datetime': '2022-08-12 18:56:16.863606',
 'http_code': 200,
 'message': 'Mapset <futures_test> successfully created.',
 'process_chain_list': [{'1': {'flags': 'l', 'module': 'g.mapsets'}}],
 'process_log': [{'executable': 'g.mapsets',
                  'id': '1',
                  'parameter': ['-l'],
                  'return_code': 0,
                  'run_time': 0.05020475387573242,
                  'stderr': ['Available mapsets:', ''],
                  'stdout': 'PERMANENT jupyter\n'}],
 'process_results': {},
 'progress': {'num_of_steps': 1, 'step': 1},
 'resource_id'

In [5]:
!ace location="futures_triangle_nc" -m

['PERMANENT', 'jupyter']


## Get Mapset info

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/test_mapset/info/" | jq

## Get Population Data

In [4]:
!curl -X GET "https://api.census.gov/data/2021/pep/population?get=POP_2021,NAME,STATE&for=county:*&key=${CENSUS_API_KEY}" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    46  100    46    0     0    269      0 --:--:-- --:--:-- --:--:--   269
parse error: Invalid numeric literal at line 1, column 6


In [3]:
!curl -X GET "https://api.census.gov/data/2017/popproj/pop?get=POP,YEAR&for=us:1&key=${CENSUS_API_KEY}" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    45    0    45    0     0     47      0 --:--:-- --:--:-- --:--:--    47
[
  [
    "POP",
    "YEAR",
    "us"
  ],
  [
    "3970145",
    "2016",
    "1"
  ]
]


In [64]:
!file -i "./scripts/FUTURES/data/co-est2020.csv"

!file -i "./scripts/FUTURES/data/co-est00int-tot.csv"

./scripts/FUTURES/data/co-est2020.csv: text/csv; charset=us-ascii
./scripts/FUTURES/data/co-est00int-tot.csv: text/csv; charset=us-ascii


In [95]:
def population_trends(counties, output_file, years=['2001', '2006', '2011', '2016']):
    """
    Prepares census data for as a FUTURES input
    :param (list[string]) counties List of geoids
    :param (string) output_file Where to save the output data
    :param (list[string]) years List of what years to include in output
    :returns csvfile
    
    # Example Output
    # FIPS,37063,37135,37183
    # 2001,229249,118826,660434
    # 2006,246323,124593,786754
    # 2011,275326,135016,924256
    # 2016,303289,142364,1030326
    """
    fieldnames = ['FIPS'] + counties 
    output = []
    
    def _geoid(state_fip, county_fip):
       
        if len(state_fip) == 1:
            state_fip = f"0{state_fip}"
            
        county_fip_len = len(county_fip)
            
        if county_fip_len == 1:
            county_fip = f"00{county_fip}"
        elif county_fip_len == 2:
            county_fip = f"0{county_fip}"
            
        geoid = f"{state_fip}{county_fip}"
        return geoid
        
    
    with open('./scripts/FUTURES/inputs/co-est00int-tot.csv', newline='', encoding='utf-8') as csvfile:
        census0010_reader = csv.DictReader(csvfile, delimiter=',')
        for row in census0010_reader:
            geoid = _geoid(row['STATE'], row['COUNTY'])
            # print(geoid)
            if geoid in counties:
                output.append({
                    "geoid": geoid,
                    "2001": row['POPESTIMATE2001'],
                    "2002": row['POPESTIMATE2002'],
                    "2003": row['POPESTIMATE2003'],
                    "2004": row['POPESTIMATE2004'],
                    "2005": row['POPESTIMATE2005'],
                    "2006": row['POPESTIMATE2006'],
                    "2007": row['POPESTIMATE2007'],
                    "2008": row['POPESTIMATE2008'],
                    "2009": row['POPESTIMATE2009'],
                    "2010": row['POPESTIMATE2010']
                })
                
    
    with open('./scripts/FUTURES/inputs/co-est2020.csv', newline='', encoding='utf-8') as csvfile:
        census1120_reader = csv.DictReader(csvfile, delimiter=',')
        for row in census1120_reader:
            geoid = _geoid(row['STATE'], row['COUNTY'])
            if geoid in counties:
                output.append({
                    "geoid": geoid,
                    "2011": row['POPESTIMATE2011'],
                    "2012": row['POPESTIMATE2012'],
                    "2013": row['POPESTIMATE2013'],
                    "2014": row['POPESTIMATE2014'],
                    "2015": row['POPESTIMATE2015'],
                    "2016": row['POPESTIMATE2016'],
                    "2017": row['POPESTIMATE2017'],
                    "2018": row['POPESTIMATE2018'],
                    "2019": row['POPESTIMATE2019'],
                    "2020": row['POPESTIMATE2020']
                })
    
    output_rows = []
    for year in years:
        # Create new row
        row = {'FIPS': year}
        for o in output:
            # check if output contains year data
            if year in o:
                # add year data to row
                row[o['geoid']] = o[year]
        output_rows.append(row)
   
            
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(output_rows)
        
        
population_trends(
    counties=["37063", "37135", "37183"],
    years=['2001', '2006', '2011', '2016'],
    output_file='./scripts/FUTURES/inputs/population_trends.csv'
)

## 1. Load County Boundaries

In [111]:
# !ace location="CONUS" mapset="dallas_fort_worth_area" script="./scripts/FUTURES/load_counties.sh" -d
PCHAIN = !ace location="CONUS" mapset="dallas_fort_worth_area" script="./scripts/FUTURES/load_counties.sh" -d
url =  "http://localhost:8088/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=json.loads(str("".join(PCHAIN))),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-10-22 23:10:48.785029', 'accept_timestamp': 1666480248.7850277, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async'}, 'datetime': '2022-10-22 23:10:48.787037', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'resource_id': 'resource_id-40addd90-5a29-4984-ac9e-274360d00a1e', 'status': 'accepted', 'time_delta': 0.0020151138305664062, 'timestamp': 1666480248.7870367, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-40addd90-5a29-4984-ac9e-274360d00a1e'}, 'user_id': 'actinia-gdi'}


In [114]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   964  100   964    0     0   2688      0 --:--:-- --:--:-- --:--:--  2685
{
  "accept_datetime": "2022-10-22 23:10:48.785029",
  "accept_timestamp": 1666480248.7850277,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async"
  },
  "datetime": "2022-10-22 23:20:01.849967",
  "http_code": 200,
  "message": "Running executable r.futures.devpressure with parameters ['input=urban_2019_nonull', 'method=gravity', 'siz ... rocs=1', 'output=devpressure_30_05_01_2019', '-n'] for 135.476 seconds",
  "process_chain_list": [],
  "progress": {
    "num_of_steps": 43,
    "step": 37
  },
  "resource_id": "resource_id-

In [87]:
!ace -l
!g.extension extension=importer url=https://github.com/mundialis/importer
# !importer --help

['nc_spm_08',
 'latlong_wgs84',
 'utm32n',
 'futures_triangle_nc',
 'CONUS',
 'test_location',
 'dummy']
/home/coreywhite/.local/grass83/scripts/g.extension:167: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree
Fetching <importer> from
<https://github.com/mundialis/importer/archive/master.zip> (be patient)...
Compiling...
Installing...
Updating extensions metadata file...
Updating extension modules metadata file...
Installation of <importer> successfully finished


In [34]:
!ace location="CONUS" -m

['PERMANENT',
 'dallas_fortworth',
 'hydro',
 'nc_research_triangle',
 'seattle_area',
 'washington_dc_area',
 'western_nc']


In [23]:
!ace location="CONUS" mapset="dallas_fortworth" render_vector='counties@dallas_fortworth'

Trying to render vector_layers map counties of mapset dallas_fortworth


In [46]:
!ace location="CONUS" mapset="dallas_fort_worth_area" -v
!ace location="CONUS" mapset="dallas_fort_worth_area" -r

['counties']
[]


## 1. Prepare Data

In [47]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/data_prep.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'v.to.rast', 'id': 'v.to.rast_1804289383', 'inputs': [{'param': 'input', 'value': 'protected_areas'}, {'param': 'layer', 'value': '1'}, {'param': 'type', 'value': 'point,line,area'}, {'param': 'use', 'value': 'val'}, {'param': 'value', 'value': '1'}, {'param': 'memory', 'value': '300'}], 'outputs': [{'param': 'output', 'value': 'protected_areas'}]}, {'module': 'r.null', 'id': 'r.null_1804289383', 'inputs': [{'param': 'map', 'value': 'protected_areas'}, {'param': 'null', 'value': '0'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'urban_2001 = if(landuse_2001 >= 21 && landuse_2001 <= 24, 1, if(landuse_2001 == 11 || landuse_2001 >= 90 || protected_areas

## 2. Calculate Predictors

In [29]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/predictors.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.slope.aspect', 'id': 'r.slope.aspect_1804289383', 'inputs': [{'param': 'elevation', 'value': 'elevation'}, {'param': 'format', 'value': 'degrees'}, {'param': 'precision', 'value': 'FCELL'}, {'param': 'zscale', 'value': '1.0'}, {'param': 'min_slope', 'value': '0.0'}, {'param': 'nprocs', 'value': '1'}, {'param': 'memory', 'value': '300'}], 'outputs': [{'param': 'slope', 'value': 'slope'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'water = if(landuse_2016 == 11, 1, null())'}, {'param': 'region', 'value': 'current'}]}, {'module': 'r.grow.distance', 'id': 'r.grow.distance_1804289383', 'inputs': [{'param': 'input', 'value': 'water'}, {'param': 'metric

## 3. Development Pressure

In [131]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/development_pressure.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'urban_2001_nonull = if(isnull(urban_2001), 0, urban_2001)'}, {'param': 'region', 'value': 'current'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'urban_2016_nonull = if(isnull(urban_2016), 0, urban_2016)'}, {'param': 'region', 'value': 'current'}]}, {'module': 'r.futures.devpressure', 'id': 'r.futures.devpressure_1804289383', 'flags': 'n', 'inputs': [{'param': 'input', 'value': 'urban_2001_nonull'}, {'param': 'method', 'value': 'gravity'}, {'param': 'size', 'value': '30'}, {'param': 'gamma', 'value': '0.5'}, {'param': 'scaling_factor', 'value': '0.1'}, {'param': 

## 4. Development Potential

Now we find best model for predicting urbanization using [r.futures.potential]((https://grass.osgeo.org/grass7/manuals/addons/r.futures.potential.html)) which wraps an R script. We can run R dredge function to find "best" model. We can specify minimum and maximum number of predictors the final model should use.
Because this can be time consuming, for this tutorial we preselected one combination of predictors.


FUTURES site suitability modeled by r.futures.potential and computed by [r.futures.potsurface](https://grass.osgeo.org/grass7/manuals/addons/r.futures.potsurface.html).
We can now visualize the suitability surface using module r.futures.potsurface. It creates initial development potential (suitability) raster from predictors and model coefficients and serves only for evaluating development Potential model. The values of the resulting raster range from 0 to 1.

In [132]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/development_potential.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.futures.potential', 'id': 'r.futures.potential_1804289383', 'inputs': [{'param': 'input', 'value': 'sampling'}, {'param': 'separator', 'value': 'comma'}, {'param': 'columns', 'value': 'devpressure_30_05_01_2001,road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km'}, {'param': 'developed_column', 'value': 'urban_change_clip'}, {'param': 'subregions_column', 'value': 'counties'}, {'param': 'min_variables', 'value': '1'}, {'param': 'nprocs', 'value': '1'}], 'outputs': [{'param': 'output', 'value': 'potential.csv'}]}, {'module': 'r.futures.potsurface', 'id': 'r.futures.potsurface_1804289383', 'inputs': [{'param': 'input', 'value': 'potential.csv'}, {'param': 'subregions', '

## 5. Demand submodel (use logarithmic)

We will use r.futures.demand which derives the population vs. development relation. The relation can be linear/logarithmic/logarithmic2/exponential/exponential approach. Look for examples of the different relations in the manual.

linear: y = A + Bx  
logarithmic: y = A + Bln(x)  
logarithmic2: y = A + B * ln(x - C)        (requires SciPy)  
exponential: y = Ae^(BX)  
exp_approach: y = (1 - e^(-A(x - B))) + C        (requires SciPy)  

The format of the input population CSV files is described in the manual. It is important to have synchronized categories of subregions and the column headers of the CSV files (in our case FIPS number). We can simply generate the list of years (for which demand is computed)with this Python code:

In [133]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/demand.sh" -d

{
  "version": "1",
  "list": [
    {
      "module": "g.region",
      "id": "g.region_1804289383",
      "inputs": [
        {
          "param": "vector",
          "value": "counties"
        },
        {
          "param": "align",
          "value": "landuse_2016"
        }
      ]
    },
    {
      "module": "r.mask",
      "id": "r.mask_1804289383",
      "inputs": [
        {
          "param": "raster",
          "value": "roads"
        },
        {
          "param": "maskcats",
          "value": "0"
        },
        {
          "param": "layer",
          "value": "1"
        }
      ]
    },
    {
      "module": "r.futures.demand",
      "id": "r.futures.demand_1804289383",
      "inputs": [
        {
          "param": "development",
          "value": "urban_2001,urban_2006,urban_2011,urban_2016"
        },
        {
          "param": "subregions",
          "value": "counties"
        },
        {
          "param": "observed_population",
          "value": "popu

In [134]:
PCHAIN = '''{
  "version": "1",
  "list": [
    {
      "module": "g.region",
      "id": "g.region_1804289383",
      "inputs": [
        {
          "param": "vector",
          "value": "counties"
        },
        {
          "param": "align",
          "value": "landuse_2016"
        }
      ]
    },
    {
      "module": "r.mask",
      "id": "r.mask_1804289383",
      "inputs": [
        {
          "param": "raster",
          "value": "roads"
        },
        {
          "param": "maskcats",
          "value": "0"
        },
        {
          "param": "layer",
          "value": "1"
        }
      ]
    },
    {
      "module": "r.futures.demand",
      "id": "r.futures.demand_1804289383",
      "inputs": [
        {
          "param": "development",
          "value": "urban_2001,urban_2006,urban_2011,urban_2016"
        },
        {
          "param": "subregions",
          "value": "counties"
        },
        {
          "param": "observed_population",
          "value": "https://raw.githubusercontent.com/ncsu-landscape-dynamics/futures-model-intro-notebook/98e21bb03884939097e800e51dd58237adfec5cc/population_trend.csv"
        },
        {
          "param": "projected_population",
          "value": "https://raw.githubusercontent.com/ncsu-landscape-dynamics/futures-model-intro-notebook/98e21bb03884939097e800e51dd58237adfec5cc/population_projection.csv"
        },
        {
          "param": "simulation_times",
          "value": "2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038"
        },
        {
          "param": "method",
          "value": "logarithmic,logarithmic2,exp_approach"
        },
        {
          "param": "separator",
          "value": "comma"
        }
      ],
      "outputs": [
        {
          "param": "plot",
          "value": "plot_demand.png"
        },
        {
          "param": "demand",
          "value": "demand.csv"
        }
      ]
    }
  ]
}'''
# "value": "PG:host=db port=5432 dbname=actinia user=actinia password=actinia"
url =  "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=json.loads(str(PCHAIN)),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-08-15 18:41:35.035730', 'accept_timestamp': 1660588895.0357285, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async'}, 'datetime': '2022-08-15 18:41:35.037443', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'resource_id': 'resource_id-3c2cd531-5ccf-4995-a764-c9d4825f97d5', 'status': 'accepted', 'time_delta': 0.001722097396850586, 'timestamp': 1660588895.0374417, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-3c2cd531-5ccf-4995-a764-c9d4825f97d5'}, 'user_id': 'actinia-gdi'}


In [137]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3976  100  3976    0     0  11779      0 --:--:-- --:--:-- --:--:-- 11798
{
  "accept_datetime": "2022-08-15 18:41:35.035730",
  "accept_timestamp": 1660588895.0357285,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
  },
  "datetime": "2022-08-15 18:41:42.214779",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "g.region_1804289383",
          "inputs": [
            {
              "param": "vector",
              "value": "counties"
            },
            {
              "param": "align",

## 6. Patch calibration (Skip for now)

We derive patches of new development by comparing historical and latest development using [r.futures.calib](https://grass.osgeo.org/grass7/manuals/addons/r.futures.calib.html).
We can keep only patches with minimum size 2 cells (1800 = 2 x 30 x 30 m). We will derive patch libraries for each county separately (flag -s).

In [138]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/patch_calibration.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.mask', 'id': 'r.mask_1804289383', 'inputs': [{'param': 'raster', 'value': 'roads'}, {'param': 'maskcats', 'value': '0'}, {'param': 'layer', 'value': '1'}]}, {'module': 'r.futures.calib', 'id': 'r.futures.calib_1804289383', 'flags': 'sl', 'inputs': [{'param': 'development_start', 'value': 'urban_2001'}, {'param': 'development_end', 'value': 'urban_2016'}, {'param': 'patch_threshold', 'value': '1800'}, {'param': 'nprocs', 'value': '1'}, {'param': 'random_seed', 'value': '1'}, {'param': 'subregions', 'value': 'counties'}, {'param': 'separator', 'value': 'comma'}], 'outputs': [{'param': 'patch_sizes', 'value': 'patches.csv'}]}, {'module': 'r.mask', 'id': 'r.mask_1804289383', 'flags': 'r', 'inputs': [{'para

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-ba6655c8-0c25-44c9-a2c0-bfd01ec2645a| jq

## 7. FUTURES simulation

Now we have all the inputs necessary for running [r.futures.pga](https://grass.osgeo.org/grass7/manuals/addons/r.futures.pga.html):

In [100]:
with open('./outputs/calib.csv') as f:
    disc_factor, comp_mean, comp_range = f.readlines()[1].split(',')[:3]

print(f"disc_factor: {disc_factor}, comp_mean: {comp_mean}, comp_range: {comp_range}")

disc_factor: 0.10, comp_mean: 0.00, comp_range: 0.10


In [172]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/futures_simulation.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.futures.pga', 'id': 'r.futures.pga_1804289383', 'inputs': [{'param': 'developed', 'value': 'urban_2016'}, {'param': 'subregions', 'value': 'counties'}, {'param': 'predictors', 'value': 'road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km'}, {'param': 'development_pressure', 'value': 'devpressure_30_05_01_2016'}, {'param': 'n_dev_neighbourhood', 'value': '30'}, {'param': 'development_pressure_approach', 'value': 'gravity'}, {'param': 'gamma', 'value': '0.5'}, {'param': 'scaling_factor', 'value': '0.1'}, {'param': 'devpot_params', 'value': 'potential.csv'}, {'param': 'demand', 'value': 'demand.csv'}, {'param': 'separator', 'value': 'comma'}, {'param': 'patch_sizes', 'val

In [155]:
PCHAIN = '''{
  "version": "1",
  "list": [
    {
      "module": "r.futures.pga",
      "id": "r.futures.pga_1804289383",
      "inputs": [
        {
          "param": "developed",
          "value": "urban_2016"
        },
        {
          "param": "subregions",raster
          "value": "counties"
        },
        {
          "param": "predictors",
          "value": "road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km"
        },
        {
          "param": "development_pressure",
          "value": "devpressure_30_05_01_2016"
        },
        {
          "param": "n_dev_neighbourhood",
          "value": "30"
        },
        {
          "param": "development_pressure_approach",
          "value": "gravity"
        },
        {
          "param": "gamma",
          "value": "0.5"
        },
        {
          "param": "scaling_factor",
          "value": "0.1"
        },
        {
          "param": "devpot_params",
          "value": "potential.csv"
        },
        {
          "param": "demand",
          "value": "demand.csv"
        },
        {
          "param": "separator",
          "value": "comma"
        },
        {
          "param": "patch_sizes",
          "value": "patches.csv"
        },
        {
          "param": "num_neighbors",
          "value": "4"
        },
        {
          "param": "discount_factor",
          "value": "0.10"
        },
        {
          "param": "seed_search",
          "value": "probability"
        },
        {
          "param": "compactness_mean",
          "value": "0.00"
        },
        {
          "param": "compactness_range",
          "value": "0.100.10,0.00,0.10,0.96,0.29,0.63"
        },
        {
          "param": "incentive_power",
          "value": "1"
        },
        {
          "param": "random_seed",
          "value": "1"
        }
      ],
      "outputs": [
        {
          "param": "output",
          "value": "final"
        },
        {
          "param": "output_series",
          "value": "step"
        }
      ]
    }
  ]
}'''
# "value": "PG:host=db port=5432 dbname=actinia user=actinia password=actinia"
url =  "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=json.loads(str(PCHAIN)),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-08-15 19:01:56.876588', 'accept_timestamp': 1660590116.8765862, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async'}, 'datetime': '2022-08-15 19:01:56.877996', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'resource_id': 'resource_id-14b84fb5-8426-4981-a749-fac4ff3f2aca', 'status': 'accepted', 'time_delta': 0.001416921615600586, 'timestamp': 1660590116.8779953, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-14b84fb5-8426-4981-a749-fac4ff3f2aca'}, 'user_id': 'actinia-gdi'}


In [163]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3589  100  3589    0     0   9514      0 --:--:-- --:--:-- --:--:--  9519
{
  "accept_datetime": "2022-08-15 19:01:56.876588",
  "accept_timestamp": 1660590116.8765862,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
  },
  "datetime": "2022-08-15 19:03:39.846986",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "r.futures.pga_1804289383",
          "inputs": [
            {
              "param": "developed",
              "value": "urban_2016"
            },
            {
              "param"

## Postprocessing of multiple stochastic runs

We can run multiple stochastic simulations with [r.futures.parallelpga](https://grass.osgeo.org/grass7/manuals/addons/r.futures.parallelpga.html):

In [171]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/post_processing.sh"
# !ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/futures_simulation.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.futures.parallelpga', 'id': 'r.futures.parallelpga_1804289383', 'inputs': [{'param': 'nprocs', 'value': '10'}, {'param': 'repeat', 'value': '5'}, {'param': 'developed', 'value': 'urban_2016'}, {'param': 'subregions', 'value': 'counties'}, {'param': 'predictors', 'value': 'road_dens_perc,forest_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km'}, {'param': 'devpot_params', 'value': 'potential.csv'}, {'param': 'development_pressure', 'value': 'devpressure_30_05_01_2016'}, {'param': 'n_dev_neighbourhood', 'value': '30'}, {'param': 'development_pressure_approach', 'value': 'gravity'}, {'param': 'gamma', 'value': '0.5'}, {'param': 'scaling_factor', 'value': '0.1'}, {'param': 'demand', 'value': 'dem

In [96]:
RESPONSE_ID = "http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-174e9279-cda5-4e25-8a83-7998bec1d07d"
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45915  100 45915    0     0   127k      0 --:--:-- --:--:-- --:--:--  128k
{
  "accept_datetime": "2022-10-22 22:28:56.043253",
  "accept_timestamp": 1666477736.0432491,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async",
    "request_url": "http://actinia-core:8088/api/v3/locations/CONUS/mapsets/dallas_fort_worth_area/processing_async"
  },
  "datetime": "2022-10-22 22:32:11.950033",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "flags": "",
          "id": "v.in.ogr_26_dallas_fort_worth_area",
          "inputs": [
            {
              "param": "input",
              "value": "PG:host=db port=5432 dbname=actinia us

In [93]:
!ace kill_job="resource_id-fd19d9d3-05f1-4882-98bb-ffb66c46deea"

{'message': 'Internal Server Error'}


In [35]:
!ace list_jobs=running

resource_id-cde4c477-2e36-4301-8a34-c75b9a508e72 running 2022-09-01 01:41:56.301147
